# Наивный классификатор Байеса

## 0 Импорт необходимых библиотек

In [1]:
from typing import Iterable, Dict, Union
from collections import Counter

import pandas as pd
import numpy as np

from sklearn import metrics
from sklearn.model_selection import train_test_split

## 1 Первое задание (случай с одним признаком)

Какова вероятность отправиться на прогулку если идёт дождь, при наличии следующих наблюдений?

```
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]
```

In [2]:
data = [
        ('солнечно', True),
        ('снег', False),
        ('облачно', False),
        ('дождь', False),
        ('солнечно', True),
        ('снег', False),
        ('облачно', True),
        ('снег', False),
        ('солнечно', False),
        ('облачно', True),
        ('снег', True),
        ('солнечно', True),
        ('дождь', False),
        ('дождь', True),
        ('облачно', True),
]

df = pd.DataFrame(data, columns=['weather', 'stroll'])
df

,weather,stroll
0,солнечно,True
1,снег,False
2,облачно,False
3,дождь,False
4,солнечно,True
5,снег,False
6,облачно,True
7,снег,False
8,солнечно,False
9,облачно,True


In [3]:
df.weather.unique()

Посчитаем вероятности "наступления" каждого значения признака

In [4]:
objects_count = df.shape[0]
p_sunny = df.loc[df['weather'] == 'солнечно'].shape[0] / objects_count
p_snow = df.loc[df['weather'] == 'снег'].shape[0] / objects_count
p_cloudly = df.loc[df['weather'] == 'облачно'].shape[0] / objects_count
p_rain = df.loc[df['weather'] == 'дождь'].shape[0] / objects_count
print(p_sunny, p_snow, p_cloudly, p_rain)
print(sum((p_sunny, p_snow, p_cloudly, p_rain)))

0.26666666666666666 0.26666666666666666 0.26666666666666666 0.2
1.0


Теперь вычислим вероятность наступления события, что мы идём гулять. И вероятность противоположного события

In [5]:
p_stroll = df.loc[df['stroll'] == True].shape[0] / objects_count
p_not_stroll = 1. - p_stroll
print(p_stroll, p_not_stroll)

0.5333333333333333 0.4666666666666667


Ещё нам нужно вычислить вероятность того, идёт ли дождь, когда мы гуляем

In [6]:
p_rain_if_stroll = df.loc[(df['weather'] == 'дождь') & (df['stroll'] == True)].shape[0] / \
    df.loc[df['stroll'] == True].shape[0]
print(p_rain_if_stroll)

0.125


Наконец, вычислим вероятность события, когда мы отправляемся на прогулку, при условии, что идёт дождь

$P(stroll | rain) =\frac{P(stroll) \cdot P(rain | stroll)} {P(rain)}$

In [7]:
p_stroll_if_rain = p_stroll * p_rain_if_stroll / p_rain
print(p_stroll_if_rain)

0.3333333333333333


## 2 Второе задание (Naive Bayes from Scratch)

Разбить данные на тренировочную и тестовую выборки.
На основе обучающей создать модель наивного байеса, используя данное соотношение:

$P(class | x_1, x_2, \dots, x_n) = P(x_1|class) \cdot P(x_2|class) \cdot ... \cdot P(x_n|class) \cdot P(class)$

На тестовой выборке посчитать accuracy, precision и recall.

Если в данных содержатся числовые признаки, то разбить их на квартили (границы: 25%, 50%, 75%  -- получится 4 бинарных признака).

### 2.1 Naive Bayes from Scratch

In [33]:
# noinspection PyMethodMayBeStatic, PyPep8Naming, PyShadowingNames
class NaiveBayesFromScratch:
    def __init__(self, num_features_cols: Iterable[int]):
        self._classes_counts: Dict[Union[str, int], int] = dict()
        self._classes_probas: Dict[Union[str, int], float] = dict()
        self._features_probas_if_class: Dict[int, Dict[int, Dict[Union[str, int], float]]] = \
            {0: dict(), 1: dict()}
        # self._features_probas_if_class: Dict[int, Dict[int, Dict[Union[str, float], float]]] = \
        #     {0: defaultdict(defaultdict(float)), 1: defaultdict(defaultdict(float))}
        self._features_bins: np.ndarray = np.array([])
        self.num_features_cols: Iterable[int] = num_features_cols

    def fit(self, X_train: np.ndarray, y_train: np.ndarray) -> None:
        self._count_classes_probas(y_train)
        X_train = self._split_numeric_features(X_train)
        self._count_features_if_class_probas(X_train, y_train)

    def _count_classes_probas(self, y_train: np.ndarray) -> None:
        self._classes_counts = Counter(y_train)
        for label, value in self._classes_counts.items():
            self._classes_probas[label] = value / len(y_train)

    def _split_numeric_features(self, X: np.ndarray) -> np.ndarray:
        """Split given numeric features into quartiles
           and modify given X by adding four new binary features
           (boundaries: 25%, 50%, 75%)"""

        if self._features_bins.size == 0:
            self._features_bins = np.percentile(X[:, self.num_features_cols],
                                                q=(0, 25, 50, 75, 100),
                                                axis=0)

        for col_idx, bins in enumerate(self._features_bins.transpose()):
            values = X[:, self.num_features_cols[col_idx]]
            for i in range(bins.size - 1):
                if i == bins.size - 1:
                    is_between = (values > bins[i]) & (values <= bins[i + 1])
                else:
                    is_between = (values >= bins[i]) & (values < bins[i + 1])
                X = np.column_stack((X, is_between))

        return np.delete(X, self.num_features_cols, axis=1)

    def _count_features_if_class_probas(self,
                                        X_train: np.ndarray,
                                        y_train: np.ndarray) -> None:
        for label in self._classes_counts:
            for feature_idx, feature in enumerate(X_train.transpose()):
                for value in np.unique(feature):
                    self._features_probas_if_class.setdefault(label, dict()).setdefault(feature_idx, dict())[value] = \
                        np.sum((X_train[:, feature_idx] == value) & (y_train == label)) / self._classes_counts[label]

    def predict(self, X_test: np.ndarray) -> Iterable[np.ndarray]:
        X_test = self._split_numeric_features(X_test)
        predicted_labels, predicted_dist = np.array([]), np.array([])
        for instance in X_test:
            predicted_label, inst_dist = self._predict_single(instance)
            predicted_dist = inst_dist if predicted_dist.size == 0 else np.row_stack((predicted_dist, inst_dist))
            predicted_labels = np.hstack((predicted_labels, predicted_label))
        return predicted_labels, predicted_dist

    def _predict_single(self, instance: np.ndarray) -> Iterable[np.ndarray]:
        labels_dist = []
        for label, label_proba in self._classes_probas.items():
            predicted_proba = label_proba
            for idx, value in enumerate(instance):
                value_proba = self._features_probas_if_class[label][idx].get(value, 10 ** (-5))
                predicted_proba *= value_proba
            labels_dist.append((label, predicted_proba))
        return np.array(max(labels_dist, key=lambda t: t[1])[0]), np.array(labels_dist)

    def accuracy_score(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return metrics.accuracy_score(y_true, y_pred)

    def precision_score(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return metrics.precision_score(y_true, y_pred)

    def recall_score(self, y_true: np.ndarray, y_pred: np.ndarray) -> float:
        return metrics.recall_score(y_true, y_pred)

    def all_scores(self, y_true: np.ndarray, y_pred: np.ndarray) -> Iterable[float]:
        """Return accuracy, precision and recall scores on given labels"""
        return self.accuracy_score(y_true, y_pred), self.precision_score(y_true, y_pred), \
               self.recall_score(y_true, y_pred)

### 2.2 Подготовка данных

In [38]:
url = 'https://raw.githubusercontent.com/otverskoj/First-steps-in-Data-Analysis/main/datasets/classification/occupancy_detection_preprocessed.csv'
names = ['date', 'temperature', 'humidity', 'light', 'co2', 'humidity_ratio', 'occupancy']
df = pd.read_csv(url, names=names, skiprows=1).drop(['date'], axis=1).sample(frac=1).reset_index(drop=True)

X, y = df.drop(["occupancy"], axis=1).values, df.loc[:, "occupancy"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)

### 2.3 Апробация на своих данных

In [39]:
num_cols = np.arange(X.shape[1])
bayes = NaiveBayesFromScratch(num_cols)
bayes.fit(X_train, y_train)
y_pred, _ = bayes.predict(X_test)
print(y_pred, y_test)
print("Accuracy: ", bayes.accuracy_score(y_test, y_pred))
print("Precision: ", bayes.precision_score(y_test, y_pred))
print("Recall: ", bayes.recall_score(y_test, y_pred))

[0. 0. 0. ... 0. 0. 0.] [0 0 0 ... 0 0 0]
Accuracy:  0.9778210116731517
Precision:  0.9337105901374293
Recall:  0.9730412805391744
